<a href="https://colab.research.google.com/github/sVanshika/music_generation_using_lstm/blob/main/music_vae_ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies

In [ ]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# to allow python to pick up the newly-installed fluidsynth lib. 
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

print(magenta.__version__)
print(tensorflow.__version__)

In [ ]:
!pip install gradio

In [ ]:
import gradio

# Creating Note Sequences

In [ ]:
note_pitch_values = {'C':60, 'C#':61, 'D':62, 'D#':63, 'E':64, 'F':65, 'F#':66, 'G':67, 'G#':68, 'A':69, 'A#':70, 'B':71,' ':0}

In [ ]:
from note_seq.protobuf import music_pb2
def create_note_sequences(notes, tempo, increment):
    song = music_pb2.NoteSequence()
    start = 0.0
    end = start+increment
    for note in notes:
        song.notes.add(
            pitch = note_pitch_values[note],
            start_time = start,
            end_time = end,
            velocity = 80
        )
        start += increment
        end += increment
    song.total_time = end
    song.tempos.add(qpm=tempo)
    return song

## Creating songs

In [ ]:
''' twinkle twinkle'''
notes = ['C','C','G','G','A','A','G','F','F','E','E','D','D','C']
tempo = 60
twinkle_twinkle = create_note_sequences(notes, tempo, 0.5)
note_seq.play_sequence(twinkle_twinkle,synth=note_seq.fluidsynth)

In [ ]:
''' happy birthday '''
notes = ['C#','C#','D#','C#','F#','F',' ','C#','C#','D#','C#','G#','F#',' ']
tempo = 60
happy_birthday = create_note_sequences(notes, tempo, 0.5)
note_seq.play_sequence(happy_birthday,synth=note_seq.fluidsynth)

In [ ]:
''' doraemon '''
notes = ['A','D','D','F#','B','F#','A','A','B','A','F#','G','F#','E']
tempo = 60
doraemon = create_note_sequences(notes, tempo, 0.5)
note_seq.play_sequence(doraemon, synth=note_seq.fluidsynth)

In [ ]:
''' jan gan man '''
notes = ['C#','D#','F','F','F','F','F','F','F','F','F','D#','F','F#']
tempo = 60
jan_gan_man = create_note_sequences(notes, tempo, 0.5)
note_seq.play_sequence(jan_gan_man, synth=note_seq.fluidsynth)

In [ ]:
notes=['E','E','E','E','E','E','E','G','C','D','E']
tempo = 60
memories = create_note_sequences(notes, tempo, 0.5)
note_seq.play_sequence(memories, synth=note_seq.fluidsynth)

# Music VAE

## Initialising the model - 2bar

In [ ]:
# installing 2-bar model
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# importing dependencies
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# initializing the model
print("Initializing")
music_vae = TrainedModel(
      configs.CONFIG_MAP['cat-mel_2bar_big'], 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')

print('Done!')


Initializing
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Restoring parameters from /content/mel_2bar_big.ckpt


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


Done!


## Creating new sequences

In [ ]:
''' Creating new sequences '''
num = int(input("Number of sequences to generate:"))
generated_sequences = music_vae.sample(n=num, length=80, temperature=1.0)

for ns in generated_sequences:
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

Number of sequences to generate:2


## Interpolating two sequences 

created on 2 bars each 

In [ ]:
''' interpolate between two sequences '''
def interpolate_songs(song1, song2):
    num_steps = 8;
    note_sequences = music_vae.interpolate(
        song1,
        song2, 
        num_steps=num_steps,
        length=16
    )
    # Concatenate them into one long sequence, with the start and end sequences at each end. 
    interpolated_seq = note_seq.sequences_lib.concatenate_sequences(note_sequences)

    note_seq.play_sequence(interpolated_seq, synth=note_seq.fluidsynth)
    note_seq.plot_sequence(interpolated_seq)

    note_seq.sequence_proto_to_midi_file(interpolated_seq, 'interpolated.mid')

    return interpolated_seq

# Gradio

In [ ]:
songs_gradio_label = ['Twinkle Twinkle', 'Happy Birthday', 'Doraemon Theme Song', 'Jan Gan Man']
songs_note_seq = [twinkle_twinkle, happy_birthday, doraemon, jan_gan_man]

In [ ]:
def interpolate_util(songs):
    song1 = songs_note_seq[songs_gradio_label.index(songs[0])]
    song2 = songs_note_seq[songs_gradio_label.index(songs[1])]

    # output_audio is the result note sequence
    output_audio = interpolate_songs(song1, song2)

    note_seq.sequence_proto_to_midi_file(output_audio, 'output.mid')
    msg = "File Ready: " + songs[0]+ " + " + songs[1]
    return msg

# Creating Random Sequences

In [ ]:
''' Creating new sequences '''
num = int(input("Number of sequences to generate:"))
generated_sequences = music_vae.sample(n=num, length=80, temperature=1.0)

for ns in generated_sequences:
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

Number of sequences to generate:2


# Interpolatiing sequences

In [ ]:
songs = gradio.inputs.CheckboxGroup(choices=songs_gradio_label, default=[], type="value", label=None)
output_text = gradio.outputs.Textbox(type="auto", label=None)

interface = gradio.Interface(fn=interpolate_util, 
                             title="Music generation using Deep Learning",
                             description="Choose any two songs to see how deep learning can mix them into one",
                             inputs=songs, 
                             outputs=output_text)
interface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://12790.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
